In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
from anytree import Node, RenderTree, findall
from sklearn.datasets import make_blobs
from sklearn.metrics.pairwise import euclidean_distances
import plotly.express as px
from plotly import io as pio 
pio.templates.default = "plotly_white"

from semi_supervised_chameleon_clustering.utils import plot_cluster, graph_sum
from semi_supervised_chameleon_clustering.make_synthetic_data import generate_synthetic_data_with_hierarchy, create_synthetic_paritally_known_label
from semi_supervised_chameleon_clustering.graph import knn_graph, knn_graph_f_distance_matrix
from semi_supervised_chameleon_clustering.chameleon_cluster import partition_phase, merge_phase
from akb_distance.AKB_distance import AKB_distance

## Generate synthetic data

In [2]:
X, y = make_blobs(n_samples=10_000, n_features=5, centers=6, random_state=4, cluster_std=3)

# Add hierarchy to the clusters
data = pd.DataFrame(X, columns=[f"feat{i}" for i in range(1, 5+1)])
data = pd.concat([data, pd.Series(y, name='true_clst_l3')], axis=1)

### Add hierarchy to the clusters

In [3]:
data['true_clst_l1'] = data['true_clst_l3'].apply(lambda x: (0, 1, 2, 3) if x in [0, 1, 2, 3] else (4, 5))
def l2(x):
    if x in [0, 1]:
        return (0, 1)
    elif x in [2, 3]:
        return (2, 3)
    elif x in [4, 5]:
        return (4, 5)
data['true_clst_l2'] = data['true_clst_l3'].apply(l2)
data = data[['feat1', 'feat2', 'feat3', 'feat4', 'feat5', 'true_clst_l1',
    'true_clst_l2', 'true_clst_l3']].copy()
data['true_clst'] = data['true_clst_l1'].astype(str) + '-' + data['true_clst_l2'].astype(str) + '-' + data['true_clst_l3'].astype(str)

def known_tag(x):
    """mapping true cluster to known tag
    60% is missing
    20% is correct at l1 level
    10% is correct at l2 level
    10% is correct at l3 level
    """
    coin = np.random.random()
    l1, l2, l3 = 'nan', 'nan', 'nan'
    if coin <= 0.6:
        pass
    elif coin <= 0.8:
        l1 = str(x['true_clst_l1'])
    elif coin <= 0.9:
        l1 = str(x['true_clst_l1'])
        l2 = str(x['true_clst_l2'])
    else:
        l1 = str(x['true_clst_l1'])
        l2 = str(x['true_clst_l2'])
        l3 = str(x['true_clst_l3'])
    tag = l1 + '-' + l2 + '-' + l3
    return tag


data['known_tag'] = data.apply(known_tag, axis=1)
data[['known_tag_l1', 'known_tag_l2', 'known_tag_l3']] = data['known_tag'].str.split('-', expand=True)
data[['known_tag_l1', 'known_tag_l2', 'known_tag_l3']] = data[['known_tag_l1', 'known_tag_l2', 'known_tag_l3']].replace('nan', np.nan)

In [4]:
cluster_col = 'true_clst_l3'
n_cluster = data[cluster_col].nunique()
fig = px.scatter(x=data.iloc[:, 0], y=data.iloc[:, 1], color=data[cluster_col], color_continuous_scale='turbo', range_color=[0, 30])
fig.update_layout(height=500, width=500)
fig.update_traces(marker=dict(size=3))
fig.update_layout(title=f'Number of clusters: {n_cluster}')
fig.show()

### Creathe Synthetic Vendor Provided Classification

The vendor provided classification has a different hierarchy than the true one and is partially wrong.

In [5]:
# part of the data is mislabeled in noisy_clst_l3
# vendor tag is based on this and has a different tree structure
data['noisy_clst_l3'] = data['true_clst_l3'].apply(lambda x: np.random.choice(data['true_clst_l3'].unique()) if np.random.random() < 0.15 else x)
data['vendor_tag_l1'] = data['noisy_clst_l3'].apply(lambda x: (1, 4, 5, 0) if x in [1, 4, 5, 0] else (2, 3))
data['vendor_tag_l2'] = data['noisy_clst_l3'].apply(lambda x: (1, 4) if x in [1, 4] else (5, 0) if x in [5, 0] else (2, 3))
def l3(x):
    """create vendor tag based on noisy cluster"""
    if x == 1:
        return (1)
    elif x == 4:
        return (4)
    elif x == 5:
        return (0, 5)
    elif x == 0:
        coin = np.random.choice([0, 1])
        if coin == 1:
            return (0, 5)
        else:
            return (0)
    elif x in [2, 3]:
        return (2, 3)
data['vendor_tag_l3'] = data['noisy_clst_l3'].apply(l3)
data['vendor_tag'] = data['vendor_tag_l1'].astype(str) + '-' + data['vendor_tag_l2'].astype(str) + '-' + data['vendor_tag_l3'].astype(str)

## Calculate Distance between Instances

In [6]:
# set tree structure
A = Node('Root')
B = Node('(1, 4, 5, 0)-1', parent=A)
C = Node('(2, 3)-1', parent=A)
D = Node('(1, 4)-2', parent=B)
E = Node('(0, 5)-2', parent=B)
F = Node('(2, 3)-2', parent=C)
G = Node('1-3', parent=D)
H = Node('4-3', parent=D)
I = Node('(0, 5)-3', parent=E)
J = Node('0-3', parent=E)
K = Node('(2, 3)-3', parent=F)

l1s = [B, C]
l2s = [D, E, F]
l3s = [G, H, I, J, K]
for pre, fill, node in RenderTree(A):
    print("%s%s" % (pre, node.name))
from anytree.exporter import DotExporter
# for line in DotExporter(A):
#     print(line)

Root
├── (1, 4, 5, 0)-1
│   ├── (1, 4)-2
│   │   ├── 1-3
│   │   └── 4-3
│   └── (0, 5)-2
│       ├── (0, 5)-3
│       └── 0-3
└── (2, 3)-1
    └── (2, 3)-2
        └── (2, 3)-3


In [7]:
# calculate distance between known tag using AKB distance
l1_nodes = findall(A, filter_=lambda node: node.depth == 1)
l2_nodes = findall(A, filter_=lambda node: node.depth == 2)
l3_nodes = findall(A, filter_=lambda node: node.depth == 3)
nodes = tuple([A]) + l1_nodes + l2_nodes + l3_nodes
node_dist_mat = pd.DataFrame(index=[node.name for node in nodes], columns=[node.name for node in nodes])
for node1 in nodes:
    for node2 in nodes:
        node_dist_mat.loc[node1.name, node2.name] = AKB_distance(node1, node2)
v3 = data['vendor_tag_l3'].astype(str).values + '-3'
vendor_dist_mat = node_dist_mat.loc[v3, v3]

In [8]:
# get distance matrix for other features using euclidean distance
# get final distance using Heterogeneous Euclidean-Overlap Metric
dist_list = []
for i in range(5):
    dist_ = euclidean_distances(data.iloc[:, [i]], data.iloc[:, [i]], squared=True)
    dist_ = dist_ / (dist_.max() - dist_[~np.eye(dist_.shape[0], dtype=bool)].min())
    dist_list.append(dist_)
dist_ = vendor_dist_mat.values ** 2
dist_ = dist_ / (dist_.max() - dist_[~np.eye(dist_.shape[0], dtype=bool)].min())
dist_list.append(dist_)

dist_mat = sum(dist_list) / 6

## Create Knn Graph from Distance Matrix

In [9]:
graph = knn_graph_f_distance_matrix(n_neighbors=30, dist_mat=dist_mat)

### Cannot-link constraints

Create cannot-link matrix based on known tags. If two instances have known different tags, they will be marked as cannot-link.  
The cannot-link matrix will be input in merge phase to avoid merging clusters that has cannot-link instances.

In [10]:
# a matrix showing whether two instances has the same tag at a certain level
same_l3 = data['known_tag_l3'].values.reshape(-1, 1) == data['known_tag_l3'].values.reshape(1, -1)
same_l2 = data['known_tag_l2'].values.reshape(-1, 1) == data['known_tag_l2'].values.reshape(1, -1)
same_l1 = data['known_tag_l1'].values.reshape(-1, 1) == data['known_tag_l1'].values.reshape(1, -1)

# cannot-link matrix: known tag is not null and known tag is different
valid_l3 = data['known_tag_l3'].notna().values
valid_l2 = data['known_tag_l2'].notna().values
valid_l1 = data['known_tag_l1'].notna().values
cl_l3 = ~same_l3 & valid_l3.reshape(-1, 1) & valid_l3.reshape(1, -1)
cl_l2 = ~same_l2 & valid_l2.reshape(-1, 1) & valid_l2.reshape(1, -1)
cl_l1 = ~same_l1 & valid_l1.reshape(-1, 1) & valid_l1.reshape(1, -1)
cl_mat = cl_l3 | cl_l2 | cl_l1

In [11]:
print(cl_mat.shape)
cl_mat

(10000, 10000)


array([[False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False],
       [False,  True, False, ..., False, False,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False,  True, ..., False, False, False]])

## Pre-label Clusters

Prelabel cluster according to partially known label.  
If a instance's cluster is known at the finest level (l3 in this case), then these cluster are excluded in the partition phase, i.e., they won't be cut into smaller subclusters.

In [12]:
# pre partition according to the known tag of the finest level
known_tag_l3_dict = data['known_tag_l3'].dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l3_dict, 'cluster')
exclude_cluster = np.unique(list(known_tag_l3_dict.values())).tolist()

cluster_idxs = pd.DataFrame({'cluster': pd.Series(nx.get_node_attributes(graph, 'cluster'))})
labeled_nodes = list(cluster_idxs.index)
known_tag_l2_dict = data['known_tag_l2'].drop(labeled_nodes).dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l2_dict, 'cluster')

cluster_idxs = pd.DataFrame({'cluster': pd.Series(nx.get_node_attributes(graph, 'cluster'))})
labeled_nodes = list(cluster_idxs.index)
known_tag_l1_dict = data['known_tag_l1'].drop(labeled_nodes).dropna().to_dict()
nx.set_node_attributes(graph, known_tag_l1_dict, 'cluster')

In [13]:
print(f'Excluded clusters: {exclude_cluster}')

Excluded clusters: ['0', '1', '2', '3', '4', '5']


In [14]:
graph_sum(graph)

,count
cluster,
"(0, 1)",348
"(0, 1, 2, 3)",1320
"(2, 3)",329
"(4, 5)",1024
0,160
1,161
2,176
3,159
4,168


In [15]:
plot_cluster(data, graph)

## Partition Phase

In [16]:
partition_phase(graph, n_cluster_final=50, exclude_cluster=exclude_cluster)

Eixst nodes without cluster. Initialize 5981 nodes to cluster -1


In [17]:
plot_cluster(data, graph)

In [18]:
graph_sum(graph)

,count
cluster,
-1,187
0,188
1,188
2,187
3,188
4,187
5,187
6,187
7,167


## Merge Phase

In [19]:
merge_phase(graph, n_cluster_final=6, cl_mat=cl_mat)

Output()

Number of clusters: 49, Merged pairs: ('3', '(2, 3)')

Number of clusters: 47, Merged pairs: (37, '0')

Number of clusters: 46, Merged pairs: ('3', 17)

Number of clusters: 45, Merged pairs: ('1', '(0, 1, 2, 3)')

...

Number of clusters: 10, Merged pairs: ('1', 6)

Number of clusters: 9, Merged pairs: ('4', 26)

Number of clusters: 8, Merged pairs: (22, 4)

Number of clusters: 7, Merged pairs: ('3', 35)

Number of clusters: 6, Merged pairs: (9, 25)

In [20]:
plot_cluster(data, graph)

In [21]:
# accuracy with constraints
data['cluster'] = pd.Series(nx.get_node_attributes(graph, 'cluster'))
replace_dict = {}
for i in data['cluster'].unique():
    replace_dict[i] = int(data[data['cluster'] == i]['known_tag_l3'].value_counts().index[0])

In [22]:
acc_wi_constraints = sum(data['cluster'].replace(replace_dict) == data['true_clst_l3']) / len(data)
print('accuracy with constraints: ', acc_wi_constraints)

accuracy with constraints:  0.9161
